In [1]:
import pandas as pd
import numpy as np
import math as m

import DataHandling

In [2]:
roadId = input('Please write the Road ID: ')

Please write the Road ID: N1


In [9]:
roads = DataHandling.readRoads(roadId)
bridges = DataHandling.readBridges(roadId)
widths = DataHandling.readWidths(roadId)
traffic = DataHandling.readTraffic(roadId)

In [11]:
widths

,road,roadId,startChainage,endChainage,width,nrLanes
0,N1,1585,0.000,0.130,29.7,8.0
1,N1,1585,0.130,0.400,14.0,4.0
2,N1,1585,0.400,1.050,28.3,8.0
3,N1,1585,1.050,5.100,16.5,5.0
4,N1,1585,5.100,6.200,16.2,5.0
5,N1,1585,6.200,6.890,16.0,5.0
6,N1,1585,6.890,7.330,26.0,8.0
7,N1,1585,7.330,7.900,28.0,8.0
8,N1,1585,7.900,8.900,14.9,4.0
9,N1,1585,8.900,9.800,15.0,4.0


In [ ]:
def formatBridgeData(dataframe = loadBridgeData()):
    dataframenoduplicates = dataframe.drop_duplicates('LRPName', keep = 'first')
    dataframemodified = dataframenoduplicates.ix[:,:10]
    return dataframemodified

In [ ]:
def mergeFiles():
    